## 1. Setup and Data Loading

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
from datetime import datetime, timedelta
import sys

# Add src to path
sys.path.append('../../src')
from data.data_loader import DataLoader

# Suppress warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

print("Libraries imported successfully")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

Libraries imported successfully
Pandas version: 2.3.3
NumPy version: 2.2.6


In [2]:
# Load datasets - use direct pandas since path issue with DataLoader
import os
os.chdir('../../')  # Go to project root
print(f"Working directory: {os.getcwd()}")
print("=" * 80)
print("LOADING INFRASTRUCTURE DATASETS")
print("=" * 80)

# Load road conditions
road_conditions = pd.read_excel(
    'data/raw/dataset_rancangan.xlsx',
    sheet_name='fct_kondisi_jalan'
)
print(f"\nRoad Conditions loaded: {road_conditions.shape}")

# Load weather data
weather = pd.read_excel(
    'data/raw/dataset_rancangan.xlsx',
    sheet_name='dim_cuaca_harian (relatif)'
)
print(f"Weather data loaded: {weather.shape}")

print("\n" + "=" * 80)
print("DATA OVERVIEW")
print("=" * 80)
print(f"\nRoad Conditions columns ({len(road_conditions.columns)}): {list(road_conditions.columns[:8])}...")
print(f"\nWeather columns ({len(weather.columns)}): {list(weather.columns[:8])}...")

Working directory: c:\Users\I5\Documents\asah-2025\capstone-project\minewise_ml
LOADING INFRASTRUCTURE DATASETS

Road Conditions loaded: (12000, 19)

Road Conditions loaded: (12000, 19)
Weather data loaded: (615, 19)

DATA OVERVIEW

Road Conditions columns (19): ['id_record_jalan', 'timestamp_utc', 'lokasi_kode', 'id_segmen_jalan', 'sumber_data', 'status_jalan', 'alasan_status', 'panjang_segmen_km']...

Weather columns (19): ['id_cuaca', 'tanggal', 'lokasi_kode', 'latitude', 'longitude', 'zona_waktu', 'hujan_mm', 'prob_hujan']...
Weather data loaded: (615, 19)

DATA OVERVIEW

Road Conditions columns (19): ['id_record_jalan', 'timestamp_utc', 'lokasi_kode', 'id_segmen_jalan', 'sumber_data', 'status_jalan', 'alasan_status', 'panjang_segmen_km']...

Weather columns (19): ['id_cuaca', 'tanggal', 'lokasi_kode', 'latitude', 'longitude', 'zona_waktu', 'hujan_mm', 'prob_hujan']...


## 2. Temporal Features Engineering

In [3]:
print("=" * 80)
print("CREATING TEMPORAL FEATURES")
print("=" * 80)

# Convert timestamp to datetime
road_conditions['timestamp'] = pd.to_datetime(road_conditions['timestamp_utc'])

# Extract temporal features
road_conditions['hour'] = road_conditions['timestamp'].dt.hour
road_conditions['day_of_week'] = road_conditions['timestamp'].dt.dayofweek  # 0=Monday, 6=Sunday
road_conditions['day_name'] = road_conditions['timestamp'].dt.day_name()
road_conditions['month'] = road_conditions['timestamp'].dt.month
road_conditions['is_weekend'] = road_conditions['day_of_week'].isin([5, 6]).astype(int)

# Create shift feature (based on hour)
def get_shift(hour):
    if 7 <= hour < 15:
        return 'Shift_1'  # 07:00 - 15:00
    elif 15 <= hour < 23:
        return 'Shift_2'  # 15:00 - 23:00
    else:
        return 'Shift_3'  # 23:00 - 07:00

road_conditions['shift'] = road_conditions['hour'].apply(get_shift)

# Cyclical encoding for hour (sin/cos transformation)
road_conditions['hour_sin'] = np.sin(2 * np.pi * road_conditions['hour'] / 24)
road_conditions['hour_cos'] = np.cos(2 * np.pi * road_conditions['hour'] / 24)

# Cyclical encoding for day_of_week
road_conditions['day_sin'] = np.sin(2 * np.pi * road_conditions['day_of_week'] / 7)
road_conditions['day_cos'] = np.cos(2 * np.pi * road_conditions['day_of_week'] / 7)

print("\nTemporal features created:")
print("  - hour, day_of_week, day_name, month")
print("  - is_weekend, shift")
print("  - hour_sin, hour_cos (cyclical)")
print("  - day_sin, day_cos (cyclical)")

# Distribution of shifts
print(f"\nShift Distribution:")
print(road_conditions['shift'].value_counts())

print(f"\nWeekend vs Weekday:")
print(road_conditions['is_weekend'].value_counts())

CREATING TEMPORAL FEATURES

Temporal features created:
  - hour, day_of_week, day_name, month
  - is_weekend, shift
  - hour_sin, hour_cos (cyclical)
  - day_sin, day_cos (cyclical)

Shift Distribution:
shift
Shift_3    8029
Shift_1    2977
Shift_2     994
Name: count, dtype: int64

Weekend vs Weekday:
is_weekend
0    8693
1    3307
Name: count, dtype: int64


## 3. Weather Integration Features

In [4]:
print("=" * 80)
print("CREATING WEATHER FEATURES")
print("=" * 80)

# Prepare weather data
weather['tanggal'] = pd.to_datetime(weather['tanggal'])
weather['date'] = weather['tanggal'].dt.date

# Find rainfall column
rainfall_col = None
for col in ['curah_hujan_mm', 'rainfall_mm', 'hujan_mm']:
    if col in weather.columns:
        rainfall_col = col
        break

if rainfall_col:
    # Aggregate daily rainfall
    daily_rainfall = weather.groupby('date')[rainfall_col].sum().reset_index()
    daily_rainfall.columns = ['date', 'daily_rainfall']
    
    # Calculate cumulative rainfall (3-day, 7-day)
    daily_rainfall = daily_rainfall.sort_values('date')
    daily_rainfall['cumulative_rainfall_3d'] = daily_rainfall['daily_rainfall'].rolling(window=3, min_periods=1).sum()
    daily_rainfall['cumulative_rainfall_7d'] = daily_rainfall['daily_rainfall'].rolling(window=7, min_periods=1).sum()
    
    # Rainfall intensity category
    daily_rainfall['rainfall_intensity'] = pd.cut(
        daily_rainfall['daily_rainfall'],
        bins=[-0.1, 0, 10, 50, 1000],
        labels=['Dry', 'Light', 'Moderate', 'Heavy']
    )
    
    # Wet condition flag (rainfall > 0)
    daily_rainfall['wet_condition_flag'] = (daily_rainfall['daily_rainfall'] > 0).astype(int)
    
    # Merge with road conditions
    road_conditions['date'] = road_conditions['timestamp'].dt.date
    road_conditions = road_conditions.merge(daily_rainfall, on='date', how='left')
    
    # Fill missing weather data
    road_conditions['daily_rainfall'] = road_conditions['daily_rainfall'].fillna(0)
    road_conditions['cumulative_rainfall_3d'] = road_conditions['cumulative_rainfall_3d'].fillna(0)
    road_conditions['cumulative_rainfall_7d'] = road_conditions['cumulative_rainfall_7d'].fillna(0)
    road_conditions['wet_condition_flag'] = road_conditions['wet_condition_flag'].fillna(0).astype(int)
    road_conditions['rainfall_intensity'] = road_conditions['rainfall_intensity'].fillna('Dry')
    
    print("\nWeather features created:")
    print("  - daily_rainfall")
    print("  - cumulative_rainfall_3d")
    print("  - cumulative_rainfall_7d")
    print("  - rainfall_intensity (Dry/Light/Moderate/Heavy)")
    print("  - wet_condition_flag")
    
    print(f"\nRainfall Intensity Distribution:")
    print(road_conditions['rainfall_intensity'].value_counts())
else:
    print("Rainfall column not found in weather data")

CREATING WEATHER FEATURES

Weather features created:
  - daily_rainfall
  - cumulative_rainfall_3d
  - cumulative_rainfall_7d
  - rainfall_intensity (Dry/Light/Moderate/Heavy)
  - wet_condition_flag

Rainfall Intensity Distribution:
rainfall_intensity
Light       4534
Moderate    2704
Dry         2530
Heavy       2232
Name: count, dtype: int64


## 4. Lag Features Engineering

In [5]:
print("=" * 80)
print("CREATING LAG FEATURES")
print("=" * 80)

# Sort by timestamp
road_conditions = road_conditions.sort_values('timestamp')

# Create lag features for speed
if 'kecepatan_aktual_km_jam' in road_conditions.columns:
    # Group by road segment for segment-specific lags
    if 'id_segmen_jalan' in road_conditions.columns:
        road_conditions['speed_lag_1d'] = road_conditions.groupby('id_segmen_jalan')['kecepatan_aktual_km_jam'].shift(1)
        road_conditions['speed_lag_7d'] = road_conditions.groupby('id_segmen_jalan')['kecepatan_aktual_km_jam'].shift(7)
        road_conditions['speed_lag_14d'] = road_conditions.groupby('id_segmen_jalan')['kecepatan_aktual_km_jam'].shift(14)
    else:
        road_conditions['speed_lag_1d'] = road_conditions['kecepatan_aktual_km_jam'].shift(1)
        road_conditions['speed_lag_7d'] = road_conditions['kecepatan_aktual_km_jam'].shift(7)
        road_conditions['speed_lag_14d'] = road_conditions['kecepatan_aktual_km_jam'].shift(14)
    
    print("\nSpeed lag features created:")
    print("  - speed_lag_1d (1 day ago)")
    print("  - speed_lag_7d (7 days ago)")
    print("  - speed_lag_14d (14 days ago)")

# Create lag features for cycle time / waktu tempuh
if 'waktu_tempuh_menit' in road_conditions.columns:
    if 'id_segmen_jalan' in road_conditions.columns:
        road_conditions['cycle_time_lag_1d'] = road_conditions.groupby('id_segmen_jalan')['waktu_tempuh_menit'].shift(1)
        road_conditions['cycle_time_lag_7d'] = road_conditions.groupby('id_segmen_jalan')['waktu_tempuh_menit'].shift(7)
    else:
        road_conditions['cycle_time_lag_1d'] = road_conditions['waktu_tempuh_menit'].shift(1)
        road_conditions['cycle_time_lag_7d'] = road_conditions['waktu_tempuh_menit'].shift(7)
    
    print("\nCycle time lag features created:")
    print("  - cycle_time_lag_1d")
    print("  - cycle_time_lag_7d")

# Speed change features
if 'speed_lag_1d' in road_conditions.columns:
    road_conditions['speed_change_1d'] = road_conditions['kecepatan_aktual_km_jam'] - road_conditions['speed_lag_1d']
    road_conditions['speed_change_7d'] = road_conditions['kecepatan_aktual_km_jam'] - road_conditions['speed_lag_7d']
    
    print("\nSpeed change features created:")
    print("  - speed_change_1d (delta from yesterday)")
    print("  - speed_change_7d (delta from 7 days ago)")

CREATING LAG FEATURES

Speed lag features created:
  - speed_lag_1d (1 day ago)
  - speed_lag_7d (7 days ago)
  - speed_lag_14d (14 days ago)

Cycle time lag features created:
  - cycle_time_lag_1d
  - cycle_time_lag_7d

Speed change features created:
  - speed_change_1d (delta from yesterday)
  - speed_change_7d (delta from 7 days ago)


## 5. Rolling Statistics Features

In [6]:
print("=" * 80)
print("CREATING ROLLING STATISTICS FEATURES")
print("=" * 80)

# Rolling window features for speed
if 'kecepatan_aktual_km_jam' in road_conditions.columns:
    if 'id_segmen_jalan' in road_conditions.columns:
        # Group by road segment
        road_conditions['speed_rolling_mean_7d'] = road_conditions.groupby('id_segmen_jalan')['kecepatan_aktual_km_jam'].transform(
            lambda x: x.rolling(window=7, min_periods=1).mean()
        )
        road_conditions['speed_rolling_std_7d'] = road_conditions.groupby('id_segmen_jalan')['kecepatan_aktual_km_jam'].transform(
            lambda x: x.rolling(window=7, min_periods=1).std()
        )
        road_conditions['speed_rolling_min_7d'] = road_conditions.groupby('id_segmen_jalan')['kecepatan_aktual_km_jam'].transform(
            lambda x: x.rolling(window=7, min_periods=1).min()
        )
        road_conditions['speed_rolling_max_7d'] = road_conditions.groupby('id_segmen_jalan')['kecepatan_aktual_km_jam'].transform(
            lambda x: x.rolling(window=7, min_periods=1).max()
        )
    else:
        road_conditions['speed_rolling_mean_7d'] = road_conditions['kecepatan_aktual_km_jam'].rolling(window=7, min_periods=1).mean()
        road_conditions['speed_rolling_std_7d'] = road_conditions['kecepatan_aktual_km_jam'].rolling(window=7, min_periods=1).std()
        road_conditions['speed_rolling_min_7d'] = road_conditions['kecepatan_aktual_km_jam'].rolling(window=7, min_periods=1).min()
        road_conditions['speed_rolling_max_7d'] = road_conditions['kecepatan_aktual_km_jam'].rolling(window=7, min_periods=1).max()
    
    # Fill NaN in std with 0
    road_conditions['speed_rolling_std_7d'] = road_conditions['speed_rolling_std_7d'].fillna(0)
    
    # Speed volatility (coefficient of variation)
    road_conditions['speed_volatility_7d'] = (
        road_conditions['speed_rolling_std_7d'] / road_conditions['speed_rolling_mean_7d']
    ).replace([np.inf, -np.inf], 0).fillna(0)
    
    print("\nRolling statistics features created:")
    print("  - speed_rolling_mean_7d (7-day average)")
    print("  - speed_rolling_std_7d (7-day standard deviation)")
    print("  - speed_rolling_min_7d (7-day minimum)")
    print("  - speed_rolling_max_7d (7-day maximum)")
    print("  - speed_volatility_7d (coefficient of variation)")

# Rolling features for cycle time
if 'waktu_tempuh_menit' in road_conditions.columns:
    if 'id_segmen_jalan' in road_conditions.columns:
        road_conditions['cycle_time_rolling_mean_7d'] = road_conditions.groupby('id_segmen_jalan')['waktu_tempuh_menit'].transform(
            lambda x: x.rolling(window=7, min_periods=1).mean()
        )
    else:
        road_conditions['cycle_time_rolling_mean_7d'] = road_conditions['waktu_tempuh_menit'].rolling(window=7, min_periods=1).mean()
    
    print("\nCycle time rolling feature created:")
    print("  - cycle_time_rolling_mean_7d")

CREATING ROLLING STATISTICS FEATURES

Rolling statistics features created:
  - speed_rolling_mean_7d (7-day average)
  - speed_rolling_std_7d (7-day standard deviation)
  - speed_rolling_min_7d (7-day minimum)
  - speed_rolling_max_7d (7-day maximum)
  - speed_volatility_7d (coefficient of variation)

Cycle time rolling feature created:
  - cycle_time_rolling_mean_7d


## 6. Road Condition Features

In [7]:
print("=" * 80)
print("CREATING ROAD CONDITION FEATURES")
print("=" * 80)

# Friction risk score (inverse of friction index)
if 'indeks_friksi' in road_conditions.columns:
    road_conditions['friction_risk_score'] = (1 - road_conditions['indeks_friksi']) * 100
    print("\nFriction risk score created (higher = more dangerous)")

# Water depth category
if 'kedalaman_air_cm' in road_conditions.columns:
    road_conditions['water_depth_category'] = pd.cut(
        road_conditions['kedalaman_air_cm'],
        bins=[-0.1, 0, 5, 10, 1000],
        labels=['Dry', 'Shallow', 'Moderate', 'Deep']
    )
    print("\nWater depth category created")
    print(road_conditions['water_depth_category'].value_counts())

# Slope category
if 'kemiringan_pct' in road_conditions.columns:
    road_conditions['slope_category'] = pd.cut(
        road_conditions['kemiringan_pct'],
        bins=[-0.1, 5, 10, 100],
        labels=['Flat', 'Moderate', 'Steep']
    )
    print("\nSlope category created")
    print(road_conditions['slope_category'].value_counts())

# Speed drop percentage
if 'batas_kecepatan_km_jam' in road_conditions.columns and 'kecepatan_aktual_km_jam' in road_conditions.columns:
    road_conditions['speed_drop_pct'] = (
        (road_conditions['batas_kecepatan_km_jam'] - road_conditions['kecepatan_aktual_km_jam']) /
        road_conditions['batas_kecepatan_km_jam'] * 100
    ).clip(lower=0)  # Only positive drops
    print("\nSpeed drop percentage created")

# Road condition score (composite)
if all(col in road_conditions.columns for col in ['friction_risk_score', 'kedalaman_air_cm', 'kemiringan_pct']):
    # Normalize components to 0-1 scale
    friction_norm = road_conditions['friction_risk_score'] / 100
    water_norm = road_conditions['kedalaman_air_cm'] / road_conditions['kedalaman_air_cm'].max()
    slope_norm = road_conditions['kemiringan_pct'] / road_conditions['kemiringan_pct'].max()
    
    # Composite score (weighted average)
    road_conditions['road_hazard_score'] = (
        0.4 * friction_norm + 
        0.3 * water_norm + 
        0.3 * slope_norm
    ) * 100
    
    print("\nRoad hazard composite score created (0-100 scale)")

CREATING ROAD CONDITION FEATURES

Friction risk score created (higher = more dangerous)

Water depth category created
water_depth_category
Shallow     6959
Dry         4900
Moderate     141
Deep           0
Name: count, dtype: int64

Slope category created
slope_category
Flat        6383
Moderate    5617
Steep          0
Name: count, dtype: int64

Speed drop percentage created

Road hazard composite score created (0-100 scale)

Water depth category created
water_depth_category
Shallow     6959
Dry         4900
Moderate     141
Deep           0
Name: count, dtype: int64

Slope category created
slope_category
Flat        6383
Moderate    5617
Steep          0
Name: count, dtype: int64

Speed drop percentage created

Road hazard composite score created (0-100 scale)


## 7. Interaction Features

In [8]:
print("=" * 80)
print("CREATING INTERACTION FEATURES")
print("=" * 80)

# Rainfall x Friction interaction
if 'daily_rainfall' in road_conditions.columns and 'indeks_friksi' in road_conditions.columns:
    road_conditions['rainfall_friction_interaction'] = (
        road_conditions['daily_rainfall'] * (1 - road_conditions['indeks_friksi'])
    )
    print("\nRainfall x Friction interaction created")

# Slope x Water depth interaction
if 'kemiringan_pct' in road_conditions.columns and 'kedalaman_air_cm' in road_conditions.columns:
    road_conditions['slope_water_interaction'] = (
        road_conditions['kemiringan_pct'] * road_conditions['kedalaman_air_cm']
    )
    print("Slope x Water depth interaction created")

# Weekend x Rainfall interaction
if 'is_weekend' in road_conditions.columns and 'daily_rainfall' in road_conditions.columns:
    road_conditions['weekend_rain_flag'] = (
        (road_conditions['is_weekend'] == 1) & (road_conditions['daily_rainfall'] > 0)
    ).astype(int)
    print("Weekend x Rainfall flag created")

print("\nAll interaction features created!")

CREATING INTERACTION FEATURES

Rainfall x Friction interaction created
Slope x Water depth interaction created
Weekend x Rainfall flag created

All interaction features created!


## 8. Feature Summary & Validation

In [9]:
print("=" * 80)
print("FEATURE ENGINEERING SUMMARY")
print("=" * 80)

# Count feature categories
temporal_features = ['hour', 'day_of_week', 'month', 'is_weekend', 'shift', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos']
weather_features = ['daily_rainfall', 'cumulative_rainfall_3d', 'cumulative_rainfall_7d', 'rainfall_intensity', 'wet_condition_flag']
lag_features = [col for col in road_conditions.columns if 'lag' in col or 'change' in col]
rolling_features = [col for col in road_conditions.columns if 'rolling' in col or 'volatility' in col]
road_features = ['friction_risk_score', 'water_depth_category', 'slope_category', 'speed_drop_pct', 'road_hazard_score']
interaction_features = [col for col in road_conditions.columns if 'interaction' in col or 'weekend_rain' in col]

print(f"\nFeature Categories:")
print(f"  Temporal Features: {len([f for f in temporal_features if f in road_conditions.columns])}")
print(f"  Weather Features: {len([f for f in weather_features if f in road_conditions.columns])}")
print(f"  Lag Features: {len(lag_features)}")
print(f"  Rolling Features: {len(rolling_features)}")
print(f"  Road Features: {len([f for f in road_features if f in road_conditions.columns])}")
print(f"  Interaction Features: {len(interaction_features)}")

# Total features
total_original = 19  # Original columns
total_engineered = len(road_conditions.columns) - total_original

print(f"\nTotal Features:")
print(f"  Original: {total_original}")
print(f"  Engineered: {total_engineered}")
print(f"  Total: {len(road_conditions.columns)}")

# Check for missing values in new features
print(f"\nMissing Values Check:")
missing_counts = road_conditions[lag_features + rolling_features].isnull().sum()
if missing_counts.sum() > 0:
    print(f"  Features with missing values:")
    print(missing_counts[missing_counts > 0])
else:
    print(f"  No missing values in engineered features!")

# Data shape
print(f"\nFinal Dataset Shape: {road_conditions.shape}")
print(f"  Rows: {road_conditions.shape[0]:,}")
print(f"  Columns: {road_conditions.shape[1]}")

FEATURE ENGINEERING SUMMARY

Feature Categories:
  Temporal Features: 9
  Weather Features: 5
  Lag Features: 9
  Rolling Features: 6
  Road Features: 5
  Interaction Features: 3

Total Features:
  Original: 19
  Engineered: 38
  Total: 57

Missing Values Check:
  Features with missing values:
speed_lag_1d          15
speed_lag_7d         105
speed_lag_14d        210
cycle_time_lag_1d     15
cycle_time_lag_7d    105
speed_change_1d       15
speed_change_7d      105
dtype: int64

Final Dataset Shape: (12000, 57)
  Rows: 12,000
  Columns: 57


## 9. Save Processed Features

In [10]:
print("=" * 80)
print("SAVING PROCESSED FEATURES")
print("=" * 80)

# Define output paths (absolute from project root)
output_dir = Path('data/processed')
output_dir.mkdir(parents=True, exist_ok=True)
output_file = output_dir / 'infrastructure_features.csv'

# Save to CSV
road_conditions.to_csv(output_file, index=False)
print(f"\nFeatures saved to: {output_file.absolute()}")
print(f"File size: {output_file.stat().st_size / 1024 / 1024:.2f} MB")

# Also save to feature store (parquet for efficiency)
feature_store_dir = Path('data/feature_store')
feature_store_dir.mkdir(parents=True, exist_ok=True)
feature_store_file = feature_store_dir / 'infra_features.parquet'

road_conditions.to_parquet(feature_store_file, index=False, compression='snappy')
print(f"\nFeatures saved to feature store: {feature_store_file.absolute()}")
print(f"File size: {feature_store_file.stat().st_size / 1024 / 1024:.2f} MB")

print("\n" + "=" * 80)
print("INFRASTRUCTURE FEATURE ENGINEERING COMPLETE!")
print("=" * 80)

print("\nNext Steps:")
print("  1. Train Road Speed Regression model")
print("  2. Train Cycle Time Regression model")
print("  3. Train Road Risk Classification model")

print("\nOutput files:")
print(f"  - CSV: {output_file.absolute()}")
print(f"  - Parquet: {feature_store_file.absolute()}")

SAVING PROCESSED FEATURES

Features saved to: c:\Users\I5\Documents\asah-2025\capstone-project\minewise_ml\data\processed\infrastructure_features.csv
File size: 5.63 MB

Features saved to: c:\Users\I5\Documents\asah-2025\capstone-project\minewise_ml\data\processed\infrastructure_features.csv
File size: 5.63 MB

Features saved to feature store: c:\Users\I5\Documents\asah-2025\capstone-project\minewise_ml\data\feature_store\infra_features.parquet
File size: 0.62 MB

INFRASTRUCTURE FEATURE ENGINEERING COMPLETE!

Next Steps:
  1. Train Road Speed Regression model
  2. Train Cycle Time Regression model
  3. Train Road Risk Classification model

Output files:
  - CSV: c:\Users\I5\Documents\asah-2025\capstone-project\minewise_ml\data\processed\infrastructure_features.csv
  - Parquet: c:\Users\I5\Documents\asah-2025\capstone-project\minewise_ml\data\feature_store\infra_features.parquet

Features saved to feature store: c:\Users\I5\Documents\asah-2025\capstone-project\minewise_ml\data\feature_s